# Pitch Class Distributions in Turkish Classical Music 

This notebook contains...

Imports and Constants

In [1]:
from music21 import *
import os
import xml.etree.ElementTree as ET
import json

makamScoreDirectory = './MusicXML/'

This cell will remove the key signature from every song in the SymbTr-2.4.3 directory. Note the path for the SymbTr directory and that it should be in the parent directory of the location of this notebook

In [2]:
for dirPaths, dirNames, makamScores in os.walk(makamScoreDirectory):
    for makamScore in makamScores:
        tree = ET.parse(os.path.join(makamScoreDirectory, makamScore))
        root = tree.getroot()
        for att in root.iter('attributes'):
            if att.find('key'):
                att.remove(att.find('key'))
        tree.write(os.path.join(makamScoreDirectory, makamScore))

ParseError: no element found: line 1, column 0 (<string>)

Extract Pitch classes from files

In [ ]:
makams = {} # object to hold all the makam types with pitch distributions

# Iterate over all files in the dataset
for dirPaths, dirNames, makamScores in os.walk(makamScoreDirectory):
    for file in makamScores:
        # Parse file name to get makam and usul type
        fileNameSegments = file.split('--')
        makamType = fileNameSegments[0]
        formType = fileNameSegments[1]
        usulType = fileNameSegments[2]
        
        makamScore = os.path.join(makamScoreDirectory, file)
        score = converter.parse(makamScore)
        allNotes = score.flat.notes.stream()
        
        results = [] # array of pitches/durations to store the results
        for note in allNotes:
            result = {
                'name' : note.pitch.step + str(note.pitch.octave),
                'duration' : float(note.duration.quarterLength)
                # not sure if quarter length is the best way to use weighted note durations
            }
            if note.pitch.accidental:
                result['name'] += '_' + note.pitch.accidental.name
            results.append(result)
            
        if makamType not in makams:
            makams[makamType] = {
                'forms': {},
                'usuls': {},
            }

        if formType not in makams[makamType]['forms']:
            makams[makamType]['forms'][formType] = {}
        if usulType not in makams[makamType]['usuls']:
            makams[makamType]['usuls'][usulType] = {}
        
        for note in results:
            if note['name'] in makams[makamType]['forms'][formType]:
                makams[makamType]['forms'][formType][note['name']] += note['duration']
            else:
                makams[makamType]['forms'][formType][note['name']] = note['duration']
                
            if note['name'] in makams[makamType]['usuls'][usulType]:
                makams[makamType]['usuls'][usulType][note['name']] += note['duration']
            else:
                makams[makamType]['usuls'][usulType][note['name']] = note['duration']

Write the distributions to a file

In [ ]:
with open('./makam-distributions.json', "w") as fp:
    json.dump(makams, fp)